### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [4]:
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [5]:
print(school_data_complete.columns)

Index(['Student ID', 'student_name', 'gender', 'year', 'school_name',
       'reading_score', 'maths_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')


## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [21]:
# Calculate the total number of schools
total_schools = school_data_complete['school_name'].nunique()

# Calculate the total number of students
total_students = school_data_complete['Student ID'].count()

# Calculate the total budget
total_budget = school_data['budget'].sum()

# Calculate the average maths score
average_math_score = school_data_complete['maths_score'].mean()

# Calculate the average reading score
average_reading_score = school_data_complete['reading_score'].mean()

# Calculate the percentage of students with a passing maths score (50 or greater)
passing_math_score = school_data_complete[school_data_complete['maths_score'] >= 50]
percent_passing_math = (passing_math_score['Student ID'].count() / total_students) * 100

# Calculate the percentage of students with a passing reading score (50 or greater)
passing_reading_score = school_data_complete[school_data_complete['reading_score'] >= 50]
percent_passing_reading = (passing_reading_score['Student ID'].count() / total_students) * 100

# Calculate the percentage of students who passed maths and reading (% Overall Passing)
overall_passing_score = school_data_complete[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)]
overall_passing_percent = (overall_passing_score['Student ID'].count() / total_students) * 100

# Create a dataframe to hold the above results
summary_df = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Maths Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Maths": [percent_passing_math],
    "% Passing Reading": [percent_passing_reading],
    "% Overall Passing": [overall_passing_percent]
})

# Optional: give the displayed data cleaner formatting
summary_df_formatted = summary_df.copy()
summary_df_formatted["Total Budget"] = summary_df_formatted["Total Budget"].map("${:,.2f}".format)

summary_df_formatted

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [47]:
# Group by school name to calculate school-level metrics
school_grouped = school_data_complete.groupby('school_name')

# Calculate key metrics for each school
school_metrics = pd.DataFrame({
    'School Type': school_grouped['type'].first(),
    'Total Students': school_grouped['Student ID'].count(),
    'Total School Budget': school_grouped['budget'].first(),
    'Per Student Budget': school_grouped['budget'].first() / school_grouped['Student ID'].count(),
    'Average Maths Score': school_grouped['maths_score'].mean(),
    'Average Reading Score': school_grouped['reading_score'].mean(),
    '% Passing Maths': (school_data_complete[school_data_complete['maths_score'] >= 50].groupby('school_name')['Student ID'].count() / school_grouped['Student ID'].count()) * 100,
    '% Passing Reading': (school_data_complete[school_data_complete['reading_score'] >= 50].groupby('school_name')['Student ID'].count() / school_grouped['Student ID'].count()) * 100,
    '% Overall Passing': (school_data_complete[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)].groupby('school_name')['Student ID'].count() / school_grouped['Student ID'].count()) * 100
})

# Display the overview table
# Optional: give the displayed data cleaner formatting
school_metrics_formatted = school_metrics.copy()
school_metrics_formatted['Total School Budget'] = school_metrics_formatted['Total School Budget'].map("${:,.2f}".format)
school_metrics_formatted['Per Student Budget'] = school_metrics_formatted['Per Student Budget'].map("${:,.2f}".format)
school_metrics_formatted = school_metrics_formatted.style.format(precision=6)

school_metrics_formatted

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [51]:
# Sort the schools by % Overall Passing in descending order
top_performing_schools = school_metrics.sort_values('% Overall Passing', ascending=False)

# Display the top five performing schools
top_performing_schools['Total School Budget'] = top_performing_schools['Total School Budget'].map("${:,.2f}".format)
top_performing_schools['Per Student Budget'] = top_performing_schools['Per Student Budget'].map("${:,.2f}".format)
top_performing_schools_formatted = top_performing_schools.head(5).style.format(precision=6)

top_performing_schools_formatted

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [55]:
# Sort the schools by % Overall Passing in ascending order
worst_performing_schools = school_metrics.sort_values('% Overall Passing', ascending=True)

# Display the five worst-performing schools
worst_performing_schools['Total School Budget'] = worst_performing_schools['Total School Budget'].map("${:,.2f}".format)
worst_performing_schools['Per Student Budget'] = worst_performing_schools['Per Student Budget'].map("${:,.2f}".format)
worst_performing_schools_formatted = worst_performing_schools.head(5).style.format(precision=6)

worst_performing_schools_formatted

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [56]:
# Create pandas series for each year level
math_scores_9th = school_data_complete.loc[school_data_complete['year'] == 9].groupby('school_name')['maths_score'].mean()
math_scores_10th = school_data_complete.loc[school_data_complete['year'] == 10].groupby('school_name')['maths_score'].mean()
math_scores_11th = school_data_complete.loc[school_data_complete['year'] == 11].groupby('school_name')['maths_score'].mean()
math_scores_12th = school_data_complete.loc[school_data_complete['year'] == 12].groupby('school_name')['maths_score'].mean()

# Combine the series into a dataframe
average_math_scores_by_grade = pd.DataFrame({
    'Year 9': math_scores_9th,
    'Year 10': math_scores_10th,
    'Year 11': math_scores_11th,
    'Year 12': math_scores_12th
})

# Display the dataframe
average_math_scores_by_grade

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.49%,71.90%,72.37%,72.68%
Cabrera High School,72.32%,72.44%,71.01%,70.60%
Figueroa High School,68.48%,68.33%,68.81%,69.33%
Ford High School,69.02%,69.39%,69.25%,68.62%
Griffin High School,72.79%,71.09%,71.69%,71.47%
Hernandez High School,68.59%,68.87%,69.15%,68.99%
Holden High School,70.54%,75.11%,71.64%,73.41%
Huang High School,69.08%,68.53%,69.43%,68.64%
Johnson High School,69.47%,67.99%,68.64%,69.29%


## Reading Score by Year

* Perform the same operations as above for reading scores

In [57]:
# Create pandas series for each year level
reading_scores_9th = school_data_complete.loc[school_data_complete['year'] == 9].groupby('school_name')['reading_score'].mean()
reading_scores_10th = school_data_complete.loc[school_data_complete['year'] == 10].groupby('school_name')['reading_score'].mean()
reading_scores_11th = school_data_complete.loc[school_data_complete['year'] == 11].groupby('school_name')['reading_score'].mean()
reading_scores_12th = school_data_complete.loc[school_data_complete['year'] == 12].groupby('school_name')['reading_score'].mean()

# Combine the series into a dataframe
average_reading_scores_by_grade = pd.DataFrame({
    'Year 9': reading_scores_9th,
    'Year 10': reading_scores_10th,
    'Year 11': reading_scores_11th,
    'Year 12': reading_scores_12th
})

# Display the dataframe
average_reading_scores_by_grade

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.90%,70.85%,70.32%,72.20%
Cabrera High School,71.17%,71.33%,71.20%,71.86%
Figueroa High School,70.26%,67.68%,69.15%,69.08%
Ford High School,69.62%,68.99%,70.74%,68.85%
Griffin High School,72.03%,70.75%,72.39%,69.43%
Hernandez High School,68.48%,70.62%,68.42%,69.24%
Holden High School,71.60%,71.10%,73.31%,70.48%
Huang High School,68.67%,69.52%,68.74%,68.67%
Johnson High School,68.72%,69.30%,69.97%,67.99%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [37]:
# Define spending bins and labels
spending_bins = [0, 585, 630, 645, 680]
group_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Add a new column to the school_metrics DataFrame for spending ranges
school_metrics["Spending Ranges (Per Student)"] = pd.cut(school_metrics["Per Student Budget"], spending_bins, labels=group_labels)

# Group by spending ranges
spending_grouped = school_metrics.groupby("Spending Ranges (Per Student)")

# Calculate metrics for each spending range
average_math_score = spending_grouped["Average Maths Score"].mean()
average_reading_score = spending_grouped["Average Reading Score"].mean()
percent_passing_math = spending_grouped["% Passing Maths"].mean()
percent_passing_reading = spending_grouped["% Passing Reading"].mean()

# Calculate overall passing rate (average of % Passing Maths and % Passing Reading)
overall_passing_rate = (percent_passing_math + percent_passing_reading) / 2

# Create a DataFrame for the results
spending_summary = pd.DataFrame({
    "Average Maths Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Maths": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})

# Display the spending summary table
spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,71.36%,70.72%,88.84%,86.39%,87.61%
$585-630,72.07%,71.03%,91.52%,87.29%,89.41%
$630-645,69.85%,69.84%,84.69%,83.76%,84.22%
$645-680,68.88%,69.05%,81.57%,81.77%,81.67%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [38]:
# Define school size bins and labels
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Add a new column to the school_metrics DataFrame for school size
school_metrics["School Size"] = pd.cut(school_metrics["Total Students"], size_bins, labels=size_labels)

# Group by school size
size_grouped = school_metrics.groupby("School Size")

# Calculate metrics for each school size
average_math_score = size_grouped["Average Maths Score"].mean()
average_reading_score = size_grouped["Average Reading Score"].mean()
percent_passing_math = size_grouped["% Passing Maths"].mean()
percent_passing_reading = size_grouped["% Passing Reading"].mean()

# Calculate overall passing rate (average of % Passing Maths and % Passing Reading)
overall_passing_rate = (percent_passing_math + percent_passing_reading) / 2

# Create a DataFrame for the results
size_summary = pd.DataFrame({
    "Average Maths Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Maths": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})

# Display the size summary table
size_summary


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),72.34%,71.64%,90.81%,87.56%,89.18%
Medium (1000-2000),71.42%,70.72%,89.85%,86.71%,88.28%
Large (2000-5000),69.75%,69.58%,84.25%,83.30%,83.78%


## Scores by School Type

* Perform the same operations as above, based on school type

In [39]:
# Group by school type
type_grouped = school_metrics.groupby("School Type")

# Calculate metrics for each school type
average_math_score = type_grouped["Average Maths Score"].mean()
average_reading_score = type_grouped["Average Reading Score"].mean()
percent_passing_math = type_grouped["% Passing Maths"].mean()
percent_passing_reading = type_grouped["% Passing Reading"].mean()

# Calculate overall passing rate (average of % Passing Maths and % Passing Reading)
overall_passing_rate = (percent_passing_math + percent_passing_reading) / 2

# Create a DataFrame for the results
type_summary = pd.DataFrame({
    "Average Maths Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Maths": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})

# Display the type summary table
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,Overall Passing Rate
School Type,,,,,
Government,69.83%,69.68%,84.46%,83.59%,84.02%
Independent,71.37%,70.72%,89.20%,86.25%,87.73%
